In [1]:
import requests
import numpy
import pandas
import os
import base64
import certifi
import datetime
from urllib.parse import urlencode


Authentication: I prove who I say I am -> username & password

Authorization: I prove that someone else told me I could acess something -> Oauth


In [2]:
client_id = "da52de03a4e04567a9e793cad846b349"
client_secret = "95970fa08e2d4df5903a9404790994c0"


In [9]:
class Spotify_API(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    api_version = 'v1'
    base_url = 'https://api.spotify.com/{}'.format(api_version)
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs) #if you want inherit for request 
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_id == None or client_secret == None:
            raise Exception('Client Id and Secret not set')
            
        #need to encode to base 64 sting as per spotify docs
        client_creds_b64 = base64.b64encode(f"{client_id}:{client_secret}".encode())
        return client_creds_b64.decode()

    def get_token_header(self):
        client_creds_b64 = self.get_client_credentials()
        return {"Authorization": f"Basic {client_creds_b64}"}
    
    def get_token_data(self):
        return {'grant_type' : 'client_credentials'}
    
    def perform_authentication(self):
        print('Authorization')
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_header()
        
        req = requests.post(token_url, data = token_data, headers = token_headers)
        
        if req.status_code not in range(200,299):
            raise Exception('Could not authentiacate client')
        
        data = req.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] #in seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now

        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        print(token)
        
        if expires < now:
            #authentitacation expired, redo
            self.perform_authentication()

            return self.get_access_token()
        
        elif token == None:
            self.perform_authentication()
            return self.get_access_token()
            
        return token
    
    
    def get_ressources_headers(self):
        access_token = self.get_access_token()
        header = {'Authorization': f"Bearer {access_token}"}
        return header
    
    def get_ressources(self, lookup_id, ressource_type = 'artists'):
        endpoint = f"{self.base_url}/{ressource_type}/{lookup_id}"
        headers = self.get_ressources_headers()
        req = requests.get(endpoint, headers = headers)
        #print(req)
        if req.status_code not in range(200,299):
            return {}
        return req.json()
    
    def get_album(self, album_id):
        return self.get_ressources(album_id, ressource_type = 'albums')
    
    def get_artist(self, artist_id):
        return self.get_ressources(artist_id, ressource_type = 'artists')
    
    def get_track(self, track_id):
        return self.get_ressources(track_id, ressource_type = 'track')
    
    def base_search(self, query_params, search_type='track'):
        
        header = self.get_ressources_headers()
        endpoint = f"{self.base_url}/search" #get
        search_url = f"{endpoint}?{query_params}"
        print(search_url)
        
        req = requests.get(search_url, headers = header)
        if req.status_code not in range(200,299):
            return {}
        return req.json()

    def search(self, query = None, operator = None, operator_query = None, search_type = 'artist'):
        if query == None:
            raise Exception('A query is required')
        
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])#converting dict to list
            
        if operator != None and operator_query != None:
            operator = operator.upper()
            if operator == 'OR' or operator == 'NOT':
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
            
        query_params = urlencode({"q": query, "type":search_type.lower()}) #url ready string
        #print(query_params)
        
        return self.base_search(query_params)

In [10]:
spotify = Spotify_API(client_id, client_secret)

In [11]:

spotify.search(query = 'Hangman', search_type = 'track')

None
Authorization
BQCuwWfZLyGcYFjeOfDwZdhL9mylwEDz1y73-5Onrcgg7rj2RwHjO_9YdrVLME75Xn4eyFpp1joQ9NUMlfA
https://api.spotify.com/v1/search?q=Hangman&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Hangman&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6DvlemMOlMH6VD9tB7BPRf'},
       'href': 'https://api.spotify.com/v1/artists/6DvlemMOlMH6VD9tB7BPRf',
       'id': '6DvlemMOlMH6VD9tB7BPRf',
       'name': 'Tom MacDonald',
       'type': 'artist',
       'uri': 'spotify:artist:6DvlemMOlMH6VD9tB7BPRf'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
     

In [12]:
spotify.search({'track' : 'Hangman', 'artist': "Dave"}, search_type = 'track')

BQCuwWfZLyGcYFjeOfDwZdhL9mylwEDz1y73-5Onrcgg7rj2RwHjO_9YdrVLME75Xn4eyFpp1joQ9NUMlfA
https://api.spotify.com/v1/search?q=track%3AHangman+artist%3ADave&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AHangman+artist%3ADave&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6Ip8FS7vWT1uKkJSweANQK'},
       'href': 'https://api.spotify.com/v1/artists/6Ip8FS7vWT1uKkJSweANQK',
       'id': '6Ip8FS7vWT1uKkJSweANQK',
       'name': 'Dave',
       'type': 'artist',
       'uri': 'spotify:artist:6Ip8FS7vWT1uKkJSweANQK'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
    

In [13]:
spotify.search(query= 'Hangman', operator ='not', operator_query='Tom', search_type = 'track')

BQCuwWfZLyGcYFjeOfDwZdhL9mylwEDz1y73-5Onrcgg7rj2RwHjO_9YdrVLME75Xn4eyFpp1joQ9NUMlfA
https://api.spotify.com/v1/search?q=Hangman+NOT+Tom&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Hangman+NOT+Tom&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0L5fC7Ogm2YwgqVCRcF1bT'},
       'href': 'https://api.spotify.com/v1/artists/0L5fC7Ogm2YwgqVCRcF1bT',
       'id': '0L5fC7Ogm2YwgqVCRcF1bT',
       'name': 'Volbeat',
       'type': 'artist',
       'uri': 'spotify:artist:0L5fC7Ogm2YwgqVCRcF1bT'}],
     'available_markets': ['CA', 'MX', 'US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/5SBrIIYCvThaqN9r1SV2pv'},
     'href': 'https://api.spotify.com/v1/albums/5SBrIIYCvThaqN9r1SV2pv',
     'id': '5SBrIIYCvThaqN9r1SV2pv',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b27357e78d8652bc2c54332888b7',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e0257e78d8652bc2c54332888b7',
       'width': 300},
      {